In [1]:
from PIL import Image, ImageDraw, ImageOps, ImageFilter
import cv2
import numpy as np
import json
from matplotlib import pyplot as plt
from copy import deepcopy
import copy
import os
from math import ceil
import os.path
#import pycuda.driver as cuda
#import pycuda.autoinit
#from pycuda.compiler import SourceModule
import json
from collections import OrderedDict

# Searching Dataset/file

In [2]:

origin_path_prefix = '/home/shkim/Workspace/data/kitti/training/'
origin_val_path_prefix = '/home/shkim/Workspace/data/COCO_2017/val2017/'
image_prefix = 'image_2/'
label_prefix = 'label_2/'
coco_origin_path_prefix = '/home/shkim/Workspace/data/COCO_2017/'

coco_prefix = 'COCO_2017/'
kitti_prefix = 'kitti/'

coco_train_image_prefix = 'train2017/'
coco_val_image_prefix = 'val2017/'
coco_annotation_prefix = 'annotations/'


annotation_prefix = 'images_annotated/'

tmp_image_name = ''
tmp_label_name = ''

pngExtension = '.png'
txtExtension = '.txt'

move_path_prefix = '/home/shkim/Workspace/data/kitti_dataset/'

move_coco_prefix = '/home/shkim/Workspace/data/coco_dataset/'

view_iter_path = '/home/shkim/Workspace/data/kitti_dataset/image_2'

visual_path = move_path_prefix + annotation_prefix + tmp_image_name

coco_category_id = [ 'person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','fire hydrant','stop sign','parking meter','bench',
                    'bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee','skis',
                    'snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle','wine glass','cup',
                    'fork','knife','spoon','bowl','banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant',
                    'bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven','toaster','sink','refrigerator','book',
                    'clock','vase','scissors','teddy bear','hair drier','toothbrush']


# Label .txt Parsing & Dataset Label Making

In [ ]:
def img_label_projection(origin_dataset_image_name, origin_dataset_label_name, dataset_label_name, origin_list):
    a = []
    b = []
    b_tmp = []
    f = open(origin_dataset_label_name, 'r')

    a = f.read().splitlines()

    b= deepcopy(a)


    for i in range(len(b)):
        b_tmp = b[i].split(' ')
        for j in range(4):
            if j == 0:
                origin_list[0][0] = float(b_tmp[4])
                origin_list[0][1] = float(b_tmp[5])
                origin_list[0][2] = float(b_tmp[6])
                origin_list[0][3] = float(b_tmp[7])
                origin_list[1][0] = float(b_tmp[8])
                origin_list[1][1] = float(b_tmp[9])
                origin_list[1][2] = float(b_tmp[10])
                origin_list[2][0] = float(b_tmp[11])
                origin_list[2][1] = float(b_tmp[12])
                origin_list[2][2] = float(b_tmp[13])
            elif j == 1:
                #print(origin_list)       
                image, new_list = Equirectangular_Projection(origin_dataset_image_name, origin_list)
                #print(new_list)
            elif j == 2:
                if(b_tmp[0] == "DontCare"):
                    b_tmp[1] = str(donecare_list[0][0])
                    b_tmp[2] = str(donecare_list[0][1])
                    b_tmp[3] = str(donecare_list[0][2])
                    b_tmp[4] = str(new_list[0][0])
                    b_tmp[5] = str(new_list[0][1])
                    b_tmp[6] = str(new_list[0][2])
                    b_tmp[7] = str(new_list[0][3])
                    b_tmp[8] = str(donecare_list[2][0])
                    b_tmp[9] = str(donecare_list[2][1])
                    b_tmp[10] = str(donecare_list[2][2])
                    b_tmp[11] = str(donecare_list[3][0])
                    b_tmp[12] = str(donecare_list[3][1])
                    b_tmp[13] = str(donecare_list[3][2])
                    b_tmp[14] = str(donecare_list[3][3])
                    break
            elif j == 3:
                b_tmp[4] = str(new_list[0][0])
                b_tmp[5] = str(new_list[0][1])
                b_tmp[6] = str(new_list[0][2])
                b_tmp[7] = str(new_list[0][3])
                b_tmp[8] = str(new_list[1][0])
                b_tmp[9] = str(new_list[1][1])
                b_tmp[10] = str(new_list[1][2])
                b_tmp[11] = str(new_list[2][0])
                b_tmp[12] = str(new_list[2][1])
                b_tmp[13] = str(new_list[2][2])
            else:
                print("error")
        #print(b_tmp)
        b_tmp = ' '.join(b_tmp)
        b[i] = b_tmp 
        a[i] = b[i] 
        
    a = '\n'.join(a)
    
    new_f = open(dataset_label_name,'w')
    new_f.write(str(a)+'\n')

    new_f.close()
    f.close()

    return image

# Json Parsing ( AI Hub )

In [ ]:
json_data_str = ''

def json_parsing_old_bbox(json_path):
  

    old_bbox = []


    with open(json_path, 'r') as f:
        json_data = json.load(f)
        old_bbox=json_data["annotations"][0]['bbox']
        json_data["annotations"][0]['bbox'] = new_bbox
        json_data_str = str(json_data)
    
    return old_bbox



def json_parsing_data_write(json_path, new_bbox):
    state = 0
    
    with open(json_path, 'r') as f:
        json_data["annotations"][0]['bbox'] = new_bbox
        json_data_str = str(json_data)
        f.close()

        
    with open(json_path, 'w') as prj_f:
        json_data_str = json_data_str.replace("\'", "\"")
        json_dict =  json.loads(json_data_str)
        json.dump(json_dict, prj_f ,indent = 4, sort_keys=False)
        state = 1

        
    return state

In [ ]:
mod = SourceModule("""
    __global__ void calculate(float *a){
        int idx = threadIdx.x + threadIdx
        
    }                     
    """)

# Json Parsing ( COCO )

In [ ]:
# Json Parsing ( AI Hub )

json_path = '/home/shkim/Workspace/data/COCO_2017/annotations/'

json_data_str = ''

json_name = 'person_keypoints_train2017.json'

json_id_name = 'instances_val2017.json'


new_list = [0.0,0.0,0.0,0.0]

image_id_shape_list = []

annotation_bbox_list = []

tmp_id_image_name = ''

json_id_path = json_path + json_id_name

json_move_coco_name = ''

move_coco_name = ''

origin_coco_train_name = '' 
cnt = 0
#coco_origin_path_prefix + coco_train_image_prefix
origin_coco_val_name = '' 

with open(json_id_path , 'r') as f:
    #with open(json_path + json_name, 'r') as f2:
    json_id_data = json.load(f)
    #json_data = json.load(f2)
    
    
    
    
    for i in range(len(json_id_data["images"])):
        image_id_shape_list.append([json_id_data["images"][i]["id"],json_id_data["images"][i]["height"], json_id_data["images"][i]["width"]])
    
    print("make id_shape_list")
    #print(image_id_shape_list)
    
    for j in range(len(json_id_data["annotations"])):
        for i in range(len(image_id_shape_list)):
            if(image_id_shape_list[i][0] == json_id_data["annotations"][j]["image_id"]):
                annotation_bbox_list.append([json_id_data["annotations"][j]["id"],image_id_shape_list[i][1],image_id_shape_list[i][2],json_id_data["annotations"][j]["bbox"]])
                break
        
    print("make annotation_bbox_list")
    #print(annotation_bbox_list)
    
    
    for i in range(len(annotation_bbox_list)):
        annotation_bbox_list[i][3] = Equirectangular_Projection(annotation_bbox_list[i][1], annotation_bbox_list[i][2], annotation_bbox_list[i][3] )
        cnt +=1
    print("make annotation_bbox_new_list")
    #print(annotation_bbox_list)
        
    for i in range(len(annotation_bbox_list)): 
        for j in range(len(json_id_data["annotations"])):
            if(annotation_bbox_list[i][0]==json_id_data["annotations"][j]["id"]):
                json_id_data["annotations"][j]["bbox"] = annotation_bbox_list[i][3]
                cnt += 1
                break
            
    print("make annotation_bbox_new_json")

    
    tmp_path = '/home/shkim/Workspace/data/coco_dataset/annotations/test_json.json'
    
    with open(tmp_path, 'w') as prj_f:
        json.dump(json_id_data, prj_f)
        
#with open(move_coco_prefix + json_name, 'w') as prj_f:

#    state = 1

In [ ]:
# Json Parsing ( AI Hub )

json_path = '/home/shkim/Workspace/data/COCO_2017/annotations/'

json_data_str = ''

json_name = 'person_keypoints_train2017.json'

json_id_name = 'instances_train2017.json'


new_list = [0.0,0.0,0.0,0.0]

points_list = []

body_jpg_list = []

tmp_id_image_name = ''

json_id_path = json_path + json_id_name

json_move_coco_name = ''

move_coco_name = ''

origin_coco_train_name = '' 

#coco_origin_path_prefix + coco_train_image_prefix
origin_coco_val_name = '' 

with open(json_id_path , 'r') as f:
    #with open(json_path + json_name, 'r') as f2:
    json_id_data = json.load(f)
    #json_data = json.load(f2)
    for j in range(len(json_id_data["annotations"])):
        if(json_id_data["annotations"][j]['category_id'] == 1):
            body_jpg_list.append(json_id_data["annotations"][j]["image_id"])
        

                    #json_id_data["annotations"][j]["bbox"] = new_list
            #json_data["annotations"][0]['bbox'] = new_bbox
            #json_data_str = str(json_data)
        
        #json_move_coco_name = move_coco_prefix + coco_annotation_prefix + json_id_name
        #json.dump(json_id_data, json_move_coco_name ,indent = 4, sort_keys=False)
    print("category_id list using json parsing complete")
    
    
    for z in range(len(body_jpg_list)):
        for i in range(len(json_id_data["images"])): 
        #for j in range(len(json_id_data["annotations"])):
            if (json_id_data["images"][i]["id"] == body_jpg_list[z]):
                tmp_id_image_name = json_id_data["images"][i]["file_name"]
                
                #print(tmp_id_image_name)
                move_coco_name = move_coco_prefix + coco_val_image_prefix + tmp_id_image_name
                origin_coco_val_name = coco_origin_path_prefix + coco_val_image_prefix + tmp_id_image_name
                if os.path.isfile(move_coco_name):
                    #print('{} image file already exist ... so skeep once'.format(tmp_id_image_name))
                    pass
                else :

                    #image, new_list = Equirectangular_Projection(origin_coco_val_name, new_list)
                    image = image_to_equirectangular_projection(origin_coco_val_name, )
                    final_image = Image.fromarray(image)
                    final_image.save(move_coco_name)
                    print('{} image file generate...'.format(tmp_id_image_name))
                    
            else:
                continue
        #points_list.append(json_data["annotations"][i]["bbox"])
        #print(json_data["annotations"][i]["id"])
        
#with open(move_coco_prefix + json_name, 'w') as prj_f:

#    state = 1

In [ ]:
# Json Parsing ( AI Hub )

json_path = '/home/shkim/Workspace/data/COCO_2017/annotations/'

json_data_str = ''

json_name = 'person_keypoints_train2017.json'

json_id_name = 'instances_val2017.json'

points_list = []

tmp_id_image_name = ''

json_id_path = json_path + json_id_name


json_move_coco_name = ''

move_coco_name = ''

origin_coco_train_name = '' 

#coco_origin_path_prefix + coco_train_image_prefix
origin_coco_val_name = '' 

with open(json_id_path , 'r') as f:
    #with open(json_path + json_name, 'r') as f2:
        json_id_data = json.load(f)
        #json_data = json.load(f2)
        for i in range(len(json_id_data["images"])):
            for j in range(len(json_id_data["annotations"])):
                if (json_id_data["images"][i]["id"] == json_id_data["annotations"][j]['image_id']):
                    tmp_id_image_name = json_id_data["images"][i]["file_name"]
                    
                    #print(tmp_id_image_name)
                    move_coco_name = move_coco_prefix + coco_val_image_prefix + tmp_id_image_name
                    origin_coco_val_name = coco_origin_path_prefix + coco_val_image_prefix + tmp_id_image_name
                    
                   
                    if os.path.isfile(move_coco_name):
                        #print('{} image file already exist ... so skeep once'.format(move_coco_name))
                        pass
                    else :
                        image, new_list = Equirectangular_Projection(origin_coco_val_name, json_id_data["annotations"][j]["bbox"])
                        final_image = Image.fromarray(image)
                        final_image.save(move_coco_name)
                    #json_id_data["annotations"][j]["bbox"] = new_list
            #json_data["annotations"][0]['bbox'] = new_bbox
            #json_data_str = str(json_data)
        
        #json_move_coco_name = move_coco_prefix + coco_annotation_prefix + json_id_name
        #json.dump(json_id_data, json_move_coco_name ,indent = 4, sort_keys=False)
        print("json file dump complete")
            #points_list.append(json_data["annotations"][i]["bbox"])
            #print(json_data["annotations"][i]["id"])
        
        
    
    
        


        
#with open(move_coco_prefix + json_name, 'w') as prj_f:

#    state = 1

# Draw bbox ( Lisence MIT )

In [ ]:
def draw_bboxes(img, bbox, identities=None, offset=(0,0)):
    for i,box in enumerate(bbox):
        x1,y1,x2,y2 = [int(i) for i in box[1:]]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[1]
        y2 += offset[1]
        # box text and bar
        id = int(i)
        color = (255,0,0)
        label = '{} {}'.format(box[0], id)
        t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 2 , 2)[0]
        cv2.rectangle(img,(x1, y1),(x2,y2),color,3)
        #cv2.rectangle(img,(x1, y1),(x1+t_size[0]-12,y1+t_size[1]-6), color,-1)
        cv2.putText(img,label,(x1,y1+t_size[1]), cv2.FONT_HERSHEY_PLAIN, 2, [0,0,0], 4)
    return img 

# Searching file & Dataset Making ( coco )

In [ ]:
file_list = os.listdir(coco_origin_path_prefix + coco_val_image_prefix)
for file in file_list:
    filename, fileExtension = os.path.splitext(file)
    if fileExtension == '.jpg' :
        print('find jpg image file......', end=' ')
        #print(filename)
        #print(fileExtension)
        tmp_image_name = filename + pngExtension
        tmp_label_name = filename + fileExtension
        origin_dataset_image_name = origin_val_path_prefix + tmp_image_name
        origin_dataset_label_name = origin_val_path_prefix + tmp_label_name
        dataset_image_name = move_coco_prefix + coco_val_image_prefix + tmp_image_name
        dataset_label_name = move_coco_prefix + coco_val_image_prefix + tmp_label_name
        if os.path.isfile(dataset_image_name):
            print('{} image file already exist ... so skeep once'.format(dataset_image_name))
        else :
            image = image_to_(origin_dataset_image_name ,  )
            #image = img_label_(origin_dataset_image_name, origin_dataset_label_name, dataset_label_name, origin_list)
            final_image = Image.fromarray(image)
            final_image.save(dataset_image_name)
            print(" done")
        

# Searching file & Dataset Making ( kitti )

In [ ]:
file_list = os.listdir(origin_path_prefix + label_prefix)
for file in file_list:
    filename, fileExtension = os.path.splitext(file)
    if fileExtension == '.txt' :
        print('find label txt file......', end=' ')
        #print(filename)
        #print(fileExtension)
        tmp_image_name = filename + pngExtension
        tmp_label_name = filename + fileExtension
        origin_dataset_image_name = origin_path_prefix + image_prefix + tmp_image_name
        origin_dataset_label_name = origin_path_prefix + label_prefix + tmp_label_name
        dataset_image_name = move_path_prefix + image_prefix + tmp_image_name
        dataset_label_name = move_path_prefix + label_prefix + tmp_label_name
        if os.path.isfile(dataset_image_name):
            print('{} image file already exist ... so skeep once'.format(ataset_image_name))
        else :
            image = image_to_equirectangular_projection(origin_dataset_image_name )
            #image = img_label_projection(origin_dataset_image_name, origin_dataset_label_name, dataset_label_name, origin_list)
            final_image = Image.fromarray(image)
            final_image.save(dataset_image_name)
            print(" done")
        


# Annotating Image ( From Image )

In [ ]:
file_list = os.listdir(move_path_prefix + image_prefix)
for file in file_list:
    filename, fileExtension = os.path.splitext(file)
    if fileExtension == '.png' :
        print('find equirectangular png file' )
        
        #print(filename)
        #print(fileExtension)
        bbox =[]
        tmp_image_name = filename + pngExtension
        tmp_label_name = filename + txtExtension
        image = cv2.imread(move_path_prefix + image_prefix + tmp_image_name,cv2.IMREAD_COLOR)
        label_f = open(move_path_prefix + label_prefix + tmp_label_name, 'r')
        a = label_f .read().splitlines()
        b = deepcopy(a)

        for i in range(len(b)):
            b_tmp = b[i].split(' ')
            bbox.append([str(b_tmp[0]),float(b_tmp[4]),float(b_tmp[5]),float(b_tmp[6]),float(b_tmp[7])])
        
        img = draw_bboxes(image,bbox)
        img = Image.fromarray(img)
        img.save(move_path_prefix + annotation_prefix + tmp_image_name)

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = image_dir
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 
        
        
# Visualizing the first 12 images.
OUTPUT_PATH = visual_path # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

# TODO Test Origin

In [ ]:
tmp_path = '/home/shkim/Workspace/data/training/'


file_list = os.listdir(tmp_path + image_prefix)
for file in file_list:
    filename, fileExtension = os.path.splitext(file)
    if fileExtension == '.png' :
        print('find png file' )
        
        #print(filename)
        #print(fileExtension)
        bbox =[]
        tmp_image_name = filename + pngExtension
        tmp_label_name = filename + txtExtension
        image = cv2.imread(tmp_path + image_prefix + tmp_image_name,cv2.IMREAD_COLOR)
        label_f = open(tmp_path  + label_prefix + tmp_label_name, 'r')
        a = label_f .read().splitlines()
        b = deepcopy(a)

        for i in range(len(b)):
            b_tmp = b[i].split(' ')
            bbox.append([str(b_tmp[0]),float(b_tmp[4]),float(b_tmp[5]),float(b_tmp[6]),float(b_tmp[7])])
        
        img = draw_bboxes(image,bbox)
        img = Image.fromarray(img)
        img.save(tmp_path + annotation_prefix + tmp_image_name)

In [ ]:
test_image = '/home/shkim/Workspace/data/test/000000000077.jpg'
test_dest1 = '/home/shkim/Workspace/data/test/77equ.jpg'
test_dest2 = '/home/shkim/Workspace/data/test/77label.jpg'

image = image_to_equirectangular_projection(test_image , )
image2, new_list = Equirectangular_Projection(test_image, origin_list)
final_image = Image.fromarray(image)
final_image.save(test_dest1)
final_image2 = Image.fromarray(image2)
final_image2.save(test_dest2)

In [ ]:
json_tmp_path = '/home/shkim/Workspace/data/coco_dataset/annotations/person_keypoints_val2017.json'


file_val_data = OrderedDict()

file_val_data["info"] = {"description": "COCO 2017 Dataset","url": "http://cocodataset.org","version": "1.0","year": 2017,"contributor": "COCO Consortium","date_created": "2017/09/01"}
file_val_data["licenses"] = []
file_val_data["images"] = []
file_val_data["annotations"] = []
file_val_data["categories"] = [{"supercategory": "person","id": 1,"name": "person","keypoints": ["nose","left_eye","right_eye","left_ear","right_ear","left_shoulder","right_shoulder","left_elbow","right_elbow","left_wrist","right_wrist","left_hip","right_hip","left_knee","right_knee","left_ankle","right_ankle"],"skeleton": [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13],[6,7],[6,8],[7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]}]

state = 0

tmp_id_list = []

tmp_cnt = 0

with open(json_tmp_path , 'r') as f:
    json_id_data = json.load(f) 
    tmp_file_path = '/home/shkim/Workspace/data/coco_dataset/val2017'
    tmp_deep_list = list(copy.deepcopy(json_id_data["images"]))
    print(len(json_id_data["images"]))
    for i in range(len(json_id_data["images"])):
        file_list = os.listdir(tmp_file_path)
        #print(len(file_list))
        tmp_image_dict = json_id_data["images"][i]
        tmp_image_list = list(tmp_image_dict.values())
        #print(tmp_image_list)
        for file in file_list:
            if (file == tmp_image_list[1] ) :
                state = 1
                break
            else :
                state = 0
        if state == 0 :
            tmp_id_list.append(tmp_image_list[7])
    
    
    #print(tmp_id_list)
    for id in tmp_id_list:
        for i in range(len(json_id_data["images"])):
            tmp_image_dict = json_id_data["images"][i]
            tmp_image_list = list(tmp_image_dict.values())
            #print(tmp_image_list)
            if(id == tmp_image_list[7]):
                tmp_deep_list.remove(json_id_data["images"][i])
                #tmp_cnt+=1

    file_val_data["images"] = tmp_deep_list
    print(len(file_data["images"]))
    file_val_data["annotations"] = json_id_data["annotations"]
        
    
tmp_path = '/home/shkim/Workspace/data/coco_dataset/annotations/test_val_json.json'

with open(tmp_path, 'w') as prj_f:
    json.dump(file_val_data, prj_f)

In [40]:
json_tmp_path = '/home/shkim/Workspace/data/COCO_2017/annotations/person_keypoints_train2017.json'


file_data = OrderedDict()

file_data["info"] = {"description": "COCO 2017 Dataset","url": "http://cocodataset.org","version": "1.0","year": 2017,"contributor": "COCO Consortium","date_created": "2017/09/01"}
file_data["licenses"] = [{"url": "","id": 1,"name": ""},{"url": "","id": 2,"name": ""},{"url": ""}]
file_data["images"] = []
file_data["annotations"] = []
file_data["categories"] = [{"supercategory": "person","id": 1,"name": "person","keypoints": ["nose","left_eye","right_eye","left_ear","right_ear","left_shoulder","right_shoulder","left_elbow","right_elbow","left_wrist","right_wrist","left_hip","right_hip","left_knee","right_knee","left_ankle","right_ankle"],"skeleton": [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13],[6,7],[6,8],[7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]}]

state = 0

tmp_id_list = []

tmp_bbox_list = []

tmp_image_list = []

tmp_cnt = 0

with open(json_tmp_path , 'r') as f:
    json_id_data = json.load(f) 
    tmp_file_path = '/home/shkim/Workspace/data/COCO_2017/train2017/'
    print("total image len : ",len(json_id_data["images"]))
    for i in range(len(json_id_data["images"])):
        tmp_image_dict = json_id_data["images"][i]
        tmp_image_list = list(tmp_image_dict.values())

        tmp_image_path = tmp_file_path + tmp_image_list[1]
        move_path = move_coco_prefix + coco_train_image_prefix + tmp_image_list[1]
        
        for j in range(len(json_id_data["annotations"])):
            tmp_anno_dict = json_id_data["annotations"][j]
            tmp_anno_list = list(tmp_anno_dict.values())
            if(tmp_anno_list[5] == tmp_image_list[7]):
                tmp_bbox_list.append([tmp_image_path,move_path,tmp_image_list[7],tmp_anno_list[8],tmp_anno_list[6],tmp_anno_list[7]])
                tmp_cnt+=1
    print("generate tmp_bbox_list")            
    for i in range(len(tmp_bbox_list)):
        
        if os.path.isfile(tmp_bbox_list[i][1]):
            continue
        else:
            if tmp_bbox_list[i][5] == 1 :
                #print("image name",tmp_bbox_list[i][1])
                #print("before bbox",tmp_bbox_list[i][4])
                image, tmp_bbox_list[i][4] = Equirectangular_Projection(tmp_bbox_list[i][0], tmp_bbox_list[i][4])
                #print("after bbox",tmp_bbox_list[i][4])
                final_image = Image.fromarray(image)
                final_image.save(tmp_bbox_list[i][1])
                #print("equ image generate")
            else:
                #print("equ image generate skip")
                continue


    print("equ image generate using tmp_bbox_list")
    for i in range(len(tmp_bbox_list)):
        for j in range(len(json_id_data["annotations"])):
            tmp_anno_list = list(json_id_data["annotations"][j].values())
            #print(tmp_anno_list)
            if(tmp_bbox_list[i][3] == tmp_anno_list[8]):
                json_id_data["annotations"][j]["bbox"] = tmp_bbox_list[i][4]
                #print("change")
                break
    print("json_data change using tmp_bbox_list ")
    file_data["images"] = json_id_data["images"]
    file_data["annotations"] = json_id_data["annotations"]
        
    
tmp_path = '/home/shkim/Workspace/data/coco_dataset/annotations/test_json.json'

with open(tmp_path, 'w') as prj_f:
    json.dump(file_data, prj_f)
    

print("json_file generate ")

total image len :  118287
generate tmp_bbox_list
equ image generate using tmp_bbox_list
json_data change using tmp_bbox_list 
json_file generate 


In [ ]:
with open(json_id_path , 'r') as f:
    #with open(json_path + json_name, 'r') as f2:
    json_id_data = json.load(f)
    #json_data = json.load(f2)
    
    
    
    
    for i in range(len(json_id_data["images"])):
        image_id_shape_list.append([json_id_data["images"][i]["id"],json_id_data["images"][i]["height"], json_id_data["images"][i]["width"]])
    
    print("make id_shape_list")
    #print(image_id_shape_list)
    
    for j in range(len(json_id_data["annotations"])):
        for i in range(len(image_id_shape_list)):
            if(image_id_shape_list[i][0] == json_id_data["annotations"][j]["image_id"]):
                annotation_bbox_list.appendexit([json_id_data["annotations"][j]["id"],image_id_shape_list[i][1],image_id_shape_list[i][2],json_id_data["annotations"][j]["bbox"]])
                break
        
    print("make annotation_bbox_list")
    #print(annotation_bbox_list)
    
    
    for i in range(len(annotation_bbox_list)):
        annotation_bbox_list[i][3] = Equirectangular_Projection(annotation_bbox_list[i][1], annotation_bbox_list[i][2], annotation_bbox_list[i][3] )
        cnt +=1
    print("make annotation_bbox_new_list")
    #print(annotation_bbox_list)
        
    for i in range(len(annotation_bbox_list)): 
        for j in range(len(json_id_data["annotations"])):
            if(annotation_bbox_list[i][0]==json_id_data["annotations"][j]["id"]):
                json_id_data["annotations"][j]["bbox"] = annotation_bbox_list[i][3]
                cnt += 1
                break
            
    print("make annotation_bbox_new_json")

    
    tmp_path = '/home/shkim/Workspace/data/coco_dataset/annotations/test_json.json'
    
    with open(tmp_path, 'w') as prj_f:
        json.dump(json_id_data, prj_f)
        
#with open(move_coco_prefix + json_name, 'w') as prj_f:

#    state = 1